# Summary
 - Index: Add trusted docs to Chroma (ETL pipeline).
 - Retrieve: Query for relevant context.
 - Generate: Feed context into an LLM for final customer-facing response.

- This is the core pattern for a simple RAG (powered by embeddings + retrieval).
- Evaluations are next mandotory step

In [ ]:
!pip install chromadb openai python-dotenv

In [1]:
import os
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions
from openai import OpenAI

In [30]:



OPENAI_API_KEY="sk-"
client = OpenAI(api_key=OPENAI_API_KEY)

# or Load env vars

In [16]:


# Setup Chroma
chroma_client = chromadb.Client()


In [17]:


embedder = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small"
)

"""To load from env:
embedder = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-small"
)
"""

'To load from env:\nembedder = embedding_functions.OpenAIEmbeddingFunction(\n    api_key=os.getenv("OPENAI_API_KEY"),\n    model_name="text-embedding-3-small"\n)\n'

In [ ]:
# Delete if it already exists
# chroma_client.delete_collection("docs")


In [20]:

collection = chroma_client.create_collection(
    name="docs",
    embedding_function=embedder   # <--- IMPORTANT, Use the Same Embedder
)

In [21]:

# Add sample docs for example:
docs = [
    "Autoship saves you time by shipping your pet’s food automatically.",
    "Chewy Pharmacy helps manage pet prescriptions with vet approval."
]
embeddings = embedder(docs)  # returns a list of embeddings



In [22]:
collection.add(
    documents=docs,
    ids=["1", "2"],
    embeddings=embeddings
)
print("✅ Docs added successfully!")

✅ Docs added successfully!


### Option 1
 - Pass query as query_texts (simplest)
 - Let Chroma handle embeddings internally:

In [31]:
# Query
query = "How does Chewy help with prescriptions?"
results = collection.query(query_texts=[query], n_results=1) # n_results=1 means we fetch the top match.
print(results["documents"][0][0])



Chewy Pharmacy helps manage pet prescriptions with vet approval.


### Option 2 
 - If you want to embed manually
 - Make sure you flatten the result properly:

In [ ]:
query_embedding = embedder([query])  # returns a list with one embedding
results = collection.query(query_embeddings=query_embedding, n_results=1) # n_results=1 means we fetch the top match.

print(results["documents"][0][0])


Chewy Pharmacy helps manage pet prescriptions with vet approval.


In [28]:
context = results["documents"][0][0]
rag_prompt = f"Keep answer in bulleted list format. in simple language. Context: {context}\n\nQuestion: {query}\nAnswer:"

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": rag_prompt}]
)

print(resp.choices[0].message.content)

- Chewy Pharmacy fills pet prescriptions.
- Requires approval from your veterinarian.
- Easy online ordering through their website or app.
- Offers a range of medications for pets.
- Can deliver prescriptions straight to your home.
- Provides reminders for when to refill prescriptions.
- Customer support is available for any questions.
- Often offers competitive pricing on pet medications.
